## Measuring VNC length

To measure the VNC length, we need to go over these steps:

1. Classify embryo orientation
1. Calculate an ROI for each image slice, based on predicted orientation
2. Measure the maximum feret diameter of each slice

> The maximum feret diameter is computed as the longest distance between points around a region's convex hull contour.

The measurement is done using the structural channel, so that the pixel intensity changes less over time, making the ROI easier to calculate.
There is a lot of fluctuation in the lengths, because the embryo flickers over time, causing the VNC to change position and deform.
Part of the change in the VNC length is that the ROI sometimes changes and accounts for regions that are actually part of the brain lobes.

To filter out the effects of movement and deformation, a baseline with a first degree polynomial was plotted with the image.

Because we are using the ROI to calculate the VNC length, the more accurate our ROI is, the better the results will be.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
from pathlib import Path

from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, vnc_length, classifier, initial_mask, find_hatching

experiment_name = '20240307'
root_dir = Path(os.getcwd()).parent
project_dir = os.path.join(root_dir, 'data', experiment_name)

In [ ]:
def get_vnc_length_ell_fit(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, first masks the image with an ellipse.'''
    img = imread(file_path)
    # This is the same as calling get_roi with window=interval
    img = img[:hp:interval,:,:]
    ell = initial_mask.fit_ellipse(img)
    mask = initial_mask.create_mask_from_ellipse(ell, img[0].shape)
    img_roi = roi.get_roi(img, window=1, mask=mask)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def get_vnc_length_from_tiff(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length using the `pasnascope` module.'''
    img = imread(file_path)
    img = img[:hp:interval,:,:]
    img_roi = roi.get_roi(img, window=1)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def avg(l):
    return sum(l)/len(l)


PIXEL_WIDTH = 1.62

img_dir = os.path.join(project_dir, 'embs')
model_path = os.path.join(project_dir, 'models', 'SVC')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')]

lengths = []

for img in imgs:
    img_path = os.path.join(img_dir, img)
    orientation = classifier.classify_image(img_path, model_path)
    
    hp = find_hatching.find_hatching_point(img_path)

    if orientation == 'v':
        vncl = get_vnc_length_from_tiff(img_path, hp, interval=20)
    elif orientation == 'l':
        vncl = get_vnc_length_ell_fit(img_path, hp, interval=20)

    lengths.append(vncl)

deltas = [(avg(l[-10:]) - avg(l[:10]))/avg(l[:10]) for l in lengths]
x = [i for i, _ in enumerate(deltas)]

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.plot(x ,deltas)

### Effect of sampling when measuring VNC length

Different intervals where selected to check how they affect the change in the VNC length.
The idea is to also see if the measurements taken manually can be directly compared with the values in here. 

In [ ]:
img_dir = os.path.join(project_dir, 'embs')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')]

i = 0
PIXEL_WIDTH = 1.62
img = imread(os.path.join(img_dir , imgs[i]), key=range(0,1000))

img_roi = roi.get_roi(img, window=1)
vnc_lengths = vnc_length.measure_VNC(img_roi)*PIXEL_WIDTH

reg = vnc_length.fit_regression(vnc_lengths)

windows = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
deltas = []
for w in windows:
    vnc_samples = vnc_lengths[::w]
    reg = vnc_length.fit_regression(vnc_samples)
    deltas.append((reg(vnc_samples.size-1)-reg(0))/reg(0))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle("% condensation x sampling interval")
ax.plot(windows, deltas, 'r:o');